# Indeed Job Scraper

## Steps

1. Get all of the URL's of EACH LISTING. for each page teh MAIN function gets, get the LINKS for that page (the job postings). For example 15 pages gets a list of 150 URLS. 
- for EACH page
    - get Mosaic job card link
    - iterate through the list of job cards
    - append list of URLS by each individual URL
    - move to next page in Indeed
2. Look through the entire URL list and process them individually.
- for each URL
- extract the parts of the page for features 
3. On each individual page, 'write a row' for the data about the job as each features.

Loop through list of URLs
for each URLs, append a list by a dictionary of its contents
dump into pd df
pd.to_pickle('filepath')

In [138]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from time import sleep
from random import randint
from datetime import datetime
import pandas as pd

In [139]:
def get_url(position, location,page_count):
        if page_count > 0:
            template = 'https://www.indeed.com/jobs?q={}&l={}&start={}'
            page_count *= 10
            url = template.format(position, location,page_count)
        else:
            template = 'https://www.indeed.com/jobs?q={}&l={}'
            url = template.format(position, location)
        
        return 'http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=' + url

def job_listings_generator(position, location,page_count=10):
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
}
    
    urls = [] 
    i = 0
    
    # POPULATE LIST OF URLs
    
    while i < page_count:
        
        url = get_url(position, location,i)  # create the url while passing in the position and location.
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find( id = "mosaic-zone-jobcards" )
        
        if cards is None:
            print('EMPTY URL HERE! ')
            print(url)
            time.sleep(6.8)
            continue
            
        cards = cards.find_all('a')
        cards_test = [card for card in cards if 'pagead' in card['href'] or '/rc/' in card['href']]
        
        for card in cards:
            record = card['href']


            if "pagead" in record:
                urls.append("https://www.indeed.com"+record)

            if "/rc/" in record:
                urls.append("https://www.indeed.com"+record)
        
        time.sleep(0.1)
        i += 1
    
    return urls


In [178]:
def job_info(url,title,location):
    try:
        response = requests.get('http://api.scraperapi.com?api_key=d7527484c16c4d0ae688e769952edb20&url='+url)
    except Exception as e:
        print(str(e))
        time.sleep(5.0)
        try:
            print('trying again!')
            response = requests.get('http://api.scraperapi.com?api_key=d7527484c16c4d0ae688e769952edb20&url='+url)
        except Exception as e:
            return {}
        
    soup = BeautifulSoup(response.text, 'html.parser')
    d = dict()
    
    
    #location
    d["location"]=location
    
    #title
    d["job_title"]=title
    
    #posting URL
    d["url"] = url

    
    job_info = soup.find(class_="jobsearch-ViewJobLayout-jobDisplay icl-Grid-col icl-u-xs-span12 icl-u-lg-span7" )
    
#     print(job_info)
    
    
    if job_info is None:
        d["job_desc"] = ""
#         time.sleep(6.8)
        return d
    
        
    cards = job_info.find_all('h2')
    
    if len(cards) == 0:
        d["job_desc"] = ""
#         time.sleep(6.8)
        return d
            

    #description
    for card in cards:
        d["job_desc"]=card.find_next('div').text.replace("\n"," ")

    return d

In [141]:
# urls = []
# locations = ["major metros / areas"]
# titles = ["10+ job titles"]

# titles = ["data+scientist","financial+analyst", "underwriter","chemical+engineer","physician","university+recruiter"]
titles = ["physician","university+recruiter"]

locations = ["Houston","San+Diego","Dallas","San+Jose","Charlotte",'New+York','Los+Angeles','Chicago','Phoenix','Raleigh',"San+Francisco","Portland", "Seattle","Austin"]

urls = []
job_information = []


for title in titles:
    for location in locations:
        urls=job_listings_generator(title,location,11)
        for job in urls:
            job_information.append(job_info(job,title,location))
            
            time.sleep(0.1)
            
        print(f"finished {location} listings for {title}!")

finished Houston listings for physician!
EMPTY URL HERE! 
http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=https://www.indeed.com/jobs?q=physician&l=San+Diego&start=10
finished San+Diego listings for physician!
finished Dallas listings for physician!
EMPTY URL HERE! 
http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=https://www.indeed.com/jobs?q=physician&l=San+Jose&start=40
finished San+Jose listings for physician!
finished Charlotte listings for physician!
finished New+York listings for physician!
finished Los+Angeles listings for physician!
finished Chicago listings for physician!
finished Phoenix listings for physician!
finished Raleigh listings for physician!
finished San+Francisco listings for physician!
finished Portland listings for physician!
finished Seattle listings for physician!
finished Austin listings for physician!
finished Houston listings for university+recruiter!
finished San+Diego listings for university+recruiter!
finished

In [180]:


# urls = []
# locations = ["major metros / areas"]
# titles = ["10+ job titles"]

# titles = ["data+scientist","financial+analyst", "underwriter","chemical+engineer","physician","university+recruiter"]
titles = ["financial+analyst","recruiter"]
locations = ['New+York',"Dallas", "Charlotte", "San+Jose", "San+Diego", "Houston",'Los+Angeles','Chicago','Phoenix','Raleigh',"San+Francisco","Portland", "Seattle","Austin"]

urls = []
job_information = []


for title in titles:
    for location in locations:
        urls=job_listings_generator(title,location,10)
        for job in urls:
            job_information.append(job_info(job,title,location))
                        
        print(f"finished {location} listings for {title}!")

finished New+York listings for financial+analyst!
finished Dallas listings for financial+analyst!
finished Charlotte listings for financial+analyst!
finished San+Jose listings for financial+analyst!
finished San+Diego listings for financial+analyst!
finished Houston listings for financial+analyst!
EMPTY URL HERE! 
http://api.scraperapi.com?api_key=f39a27865b56e74c0423ab1b9ecf1f23&url=https://www.indeed.com/jobs?q=financial+analyst&l=Los+Angeles&start=40
finished Los+Angeles listings for financial+analyst!
finished Chicago listings for financial+analyst!
finished Phoenix listings for financial+analyst!
finished Raleigh listings for financial+analyst!
finished San+Francisco listings for financial+analyst!
finished Portland listings for financial+analyst!
finished Seattle listings for financial+analyst!
finished Austin listings for financial+analyst!
finished New+York listings for recruiter!
finished Dallas listings for recruiter!
finished Charlotte listings for recruiter!
finished San+Jo

In [182]:
df = pd.DataFrame(job_information)
df

,location,job_title,url,job_desc
0,New+York,financial+analyst,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Reporting to the Director of Accounting & Fina...
1,New+York,financial+analyst,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,
2,New+York,financial+analyst,https://www.indeed.com/rc/clk?jk=271dcd21bbb67...,
3,New+York,financial+analyst,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,A local financial client is looking for a Fina...
4,New+York,financial+analyst,https://www.indeed.com/rc/clk?jk=675406ea875bd...,ViacomCBS Streaming is the premier online cont...
...,...,...,...,...
3674,Austin,recruiter,https://www.indeed.com/rc/clk?jk=e7e47bf53a0d5...,
3675,Austin,recruiter,https://www.indeed.com/rc/clk?jk=4e8cc2047c50d...,
3676,Austin,recruiter,https://www.indeed.com/rc/clk?jk=282563ddbd95d...,Gopuff is seeking a Business Recruiter to join...
3677,Austin,recruiter,https://www.indeed.com/rc/clk?jk=8db5efc9c07be...,


In [183]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3679 entries, 0 to 3678
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   location   3679 non-null   object
 1   job_title  3679 non-null   object
 2   url        3679 non-null   object
 3   job_desc   3679 non-null   object
dtypes: object(4)
memory usage: 143.7+ KB


In [184]:
# df.to_pickle(os.getcwd()+'/webscraped_pickle')
df.to_csv('newscrape_fa_rc_2')

In [185]:
df = pd.read_csv('newscrape_fa_rc_2')

In [186]:
df = df.dropna()
df.info()
df.tail()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2468 entries, 0 to 3678
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2468 non-null   int64 
 1   location    2468 non-null   object
 2   job_title   2468 non-null   object
 3   url         2468 non-null   object
 4   job_desc    2468 non-null   object
dtypes: int64(1), object(4)
memory usage: 115.7+ KB


,Unnamed: 0,location,job_title,url,job_desc
3668,3668,Austin,recruiter,https://www.indeed.com/rc/clk?jk=1c5ba97950e6c...,"RWE Renewables Americas LLC, Austin, Vereinigt..."
3670,3670,Austin,recruiter,https://www.indeed.com/rc/clk?jk=9fd646cfc3977...,Internal Market Recruiter Job title: Healthca...
3673,3673,Austin,recruiter,https://www.indeed.com/rc/clk?jk=3c1497886534e...,Job Summary: Communicates with a professional ...
3676,3676,Austin,recruiter,https://www.indeed.com/rc/clk?jk=282563ddbd95d...,Gopuff is seeking a Business Recruiter to join...
3678,3678,Austin,recruiter,https://www.indeed.com/rc/clk?jk=07d5ec53ef83d...,13400BR People WHAT YOU'LL DO As an Experience...


In [187]:
df['job_title'].value_counts()

recruiter            1308
financial+analyst    1160
Name: job_title, dtype: int64

In [188]:
df.to_csv('newscrape_fa_rc_2')

In [177]:
'''
PROGRESS BAR!
 - how far the scraper got for each occupation
"data+scientist" - DONE  
"financial+analyst" - San Jose
"underwriter" - DONE
"chemical+engineer" - DONE 
"physician" - DONE 
"university+recruiter"
'''

'\nPROGRESS BAR!\n - how far the scraper got for each occupation\n"data+scientist" - Charlotte \n"financial+analyst" - San Jose\n"underwriter" - DONE\n"chemical+engineer" - DONE \n"physician" - Los Angeles\n"university+recruiter"\n'